In [16]:
import pandas as pd 
from pycaret.regression import *
import shap
import matplotlib.pyplot as plt

# Problem Statement
The performance of concrete is universally determined by its unaxial compressive strength, which is also closely related to other physcial properties of the concrete batch. According to the General Specifications of CEDD. It is required to conduct 1-2 concrete strength test per 100m^3 batch. However, conventional Cube Crushing Test takes too much time, usually up to 28 days or more. Therefore, it may be desirable to obtain a relationship between concrete constituent materials to find its compressive strength. 

In [ ]:
df = pd.read_csv('concrete_data.csv')

In [ ]:
df.columns

In [ ]:
df.describe().T

## Model Testing
The models that have been tested are: 

    1. Light Gradient Boositn Machine 
    2. Ensemble Learning with (Lightgbm, ExtraTree Regressor, Gradient Boosting Regressor)

In [ ]:
reg101 = setup(data=df, target='concrete_compressive_strength', session_id=1)

In [ ]:
compare_models()

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
# lightgbm_tuned = tune_model(lightgbm)

In [ ]:
# print(lightgbm_tuned)

In [ ]:
# plot_model(lightgbm_tuned)

In [ ]:
# plot_model(lightgbm_tuned, plot='error')

In [ ]:
# plot_model(lightgbm_tuned, plot='feature')

In [ ]:
predict_model(lightgbm)
# predict_model(lightgbm_tuned)

Ensemble Models 

In [ ]:
best_rmse_models_top3 = compare_models(sort = 'RMSE', n_select = 3)

In [ ]:
best_rmse_models_top3

In [ ]:
blend_models(best_rmse_models_top3)

In [ ]:
stacker = stack_models(best_rmse_models_top3)

In [ ]:
stacker

In [ ]:
plot_model(stacker, plot='residuals')

## Interpret Model

https://astrobenhart.medium.com/how-to-use-shap-with-pycaret-dc9a31278621

In [3]:
df = pd.read_csv('concrete_data.csv')

In [4]:
reg101 = setup(data=df, target='concrete_compressive_strength', session_id=1,train_size=0.8)

,Description,Value
0,Session id,1
1,Target,concrete_compressive_strength
2,Target type,Regression
3,Original data shape,"(1030, 9)"
4,Transformed data shape,"(1030, 9)"
5,Transformed train set shape,"(824, 9)"
6,Transformed test set shape,"(206, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [5]:
# top5 = compare_models(n_select=5,sort='RMSE')
# tuned_top5 = [tune_model(i, optimize='RMSE') for i in top5]
# ensem_top5 = [ensemble_model(i, n_estimators = 10, optimize='RMSE') for i in tuned_top5]
# blend = blend_models(tuned_top5, optimize='RMSE')
# blend_ensem = blend_models(ensem_top5, optimize='RMSE')
# model = automl(optimize='RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.1388,21.7581,4.6103,0.9231,0.1537,0.1120,0.4850
et,Extra Trees Regressor,3.1224,24.0036,4.8456,0.9149,0.1530,0.1109,0.0440
gbr,Gradient Boosting Regressor,3.7540,25.3993,5.0110,0.9090,0.1625,0.1303,0.0350
rf,Random Forest Regressor,3.5166,26.5611,5.0891,0.9065,0.1693,0.1283,0.0800
dt,Decision Tree Regressor,4.3412,48.0636,6.7711,0.8331,0.2212,0.1529,0.0050
ada,AdaBoost Regressor,6.3155,58.8828,7.6330,0.7890,0.2854,0.2698,0.0320
knn,K Neighbors Regressor,7.1024,86.3731,9.2318,0.6930,0.3136,0.2782,0.0050
lar,Least Angle Regression,8.0984,105.8964,10.2150,0.6219,0.3310,0.3151,0.0040
ridge,Ridge Regression,8.0984,105.8964,10.2150,0.6219,0.3310,0.3151,0.0050
lr,Linear Regression,8.0984,105.8964,10.2150,0.6219,0.3310,0.3151,0.3660


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.9555,20.4511,4.5223,0.9263,0.1864,0.1251
1,2.6732,15.3974,3.9240,0.9365,0.1469,0.1083
2,2.7165,22.8270,4.7778,0.9185,0.1553,0.1062
3,2.6545,11.4935,3.3902,0.9630,0.1589,0.1016
4,3.3121,23.3066,4.8277,0.9175,0.1436,0.1088
5,3.0655,22.0702,4.6979,0.9192,0.1466,0.0958
6,3.0071,17.1646,4.1430,0.9352,0.1295,0.0942
7,2.6074,12.6018,3.5499,0.9441,0.1325,0.0942
8,3.2889,27.8226,5.2747,0.9245,0.1752,0.1242


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,6.4109,66.3177,8.1436,0.7611,0.2983,0.2763
1,6.2414,63.5786,7.9736,0.7376,0.3352,0.3173
2,6.5097,65.6304,8.1013,0.7658,0.3262,0.3095
3,6.2619,53.0542,7.2838,0.8291,0.2975,0.2858
4,6.7177,70.2266,8.3801,0.7514,0.2565,0.2324
5,6.0170,54.4281,7.3775,0.8007,0.2502,0.2243
6,5.5954,48.6065,6.9718,0.8166,0.2275,0.1943
7,5.7908,48.3656,6.9545,0.7856,0.2814,0.2621
8,7.1701,85.1247,9.2263,0.7691,0.2913,0.2732


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


...

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.1280,18.1352,4.2585,0.9347,0.1533,0.1207
1,3.2024,20.7514,4.5554,0.9144,0.1683,0.1319
2,3.0128,23.0275,4.7987,0.9178,0.1717,0.1255
3,2.8705,13.8242,3.7181,0.9555,0.1250,0.1017
4,3.5369,24.1857,4.9179,0.9144,0.1346,0.1092
5,3.4677,19.9116,4.4622,0.9271,0.1394,0.1093
6,3.0198,16.3568,4.0444,0.9383,0.1321,0.0975
7,3.4306,19.8848,4.4592,0.9119,0.1604,0.1301
8,3.8207,32.7925,5.7265,0.9110,0.1675,0.1316


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.9641,28.8616,5.3723,0.8960,0.1958,0.1537
1,3.6473,25.3458,5.0345,0.8954,0.2105,0.1678
2,3.3941,25.3410,5.0340,0.9096,0.1930,0.1458
3,3.6486,21.8252,4.6717,0.9297,0.1684,0.1373
4,4.3230,36.7764,6.0644,0.8698,0.1669,0.1375
5,3.9848,25.2823,5.0282,0.9074,0.1480,0.1236
6,3.7193,24.5485,4.9546,0.9074,0.1491,0.1166
7,3.9020,24.9381,4.9938,0.8895,0.1779,0.1490
8,4.9002,50.2441,7.0883,0.8637,0.2052,0.1692


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.7901,26.0403,5.1030,0.9062,0.1909,0.1517
1,3.8310,28.8700,5.3731,0.8808,0.2227,0.1731
2,2.8692,23.2474,4.8216,0.9170,0.1906,0.1349
3,3.7481,25.4445,5.0443,0.9180,0.1739,0.1388
4,4.1284,39.2871,6.2679,0.8609,0.1683,0.1277
5,3.6314,26.0227,5.1012,0.9047,0.1613,0.1113
6,3.5716,24.2392,4.9233,0.9085,0.1577,0.1171
7,3.3417,21.9748,4.6877,0.9026,0.1627,0.1232
8,4.7080,52.7374,7.2621,0.8569,0.2152,0.1698


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.3065,21.3156,4.6169,0.9232,0.1632,0.1246
1,2.6811,17.2861,4.1577,0.9287,0.1593,0.1165
2,2.6455,22.8296,4.7780,0.9185,0.1686,0.1146
3,2.6356,12.8898,3.5902,0.9585,0.1217,0.0936
4,2.9577,18.2843,4.2760,0.9353,0.1171,0.0917
5,3.2155,20.7202,4.5519,0.9241,0.1323,0.0936
6,3.0507,20.0049,4.4727,0.9245,0.1406,0.0976
7,2.5454,11.7040,3.4211,0.9481,0.1267,0.0987
8,3.9460,44.5588,6.6752,0.8791,0.1870,0.1352


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.2206,19.2863,4.3916,0.9305,0.1663,0.1284
1,3.0851,19.7268,4.4415,0.9186,0.1801,0.1382
2,2.7051,21.2362,4.6083,0.9242,0.1711,0.1187
3,2.9629,15.3954,3.9237,0.9504,0.1308,0.1051
4,3.5378,25.8812,5.0874,0.9084,0.1374,0.1091
5,3.3315,18.5551,4.3076,0.9321,0.1289,0.1016
6,3.0518,17.4109,4.1726,0.9343,0.1293,0.0961
7,3.1586,17.6326,4.1991,0.9218,0.1471,0.1177
8,4.0485,39.5687,6.2904,0.8927,0.1826,0.1416


In [6]:
predict_model(model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,3.1936,21.1201,4.5957,0.9190,0.1303,0.1001


[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will b

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength,prediction_label
339,297.200012,0.000000,117.500000,174.800003,9.5,1022.799988,753.500000,3,21.910000,23.189181
244,238.100006,0.000000,94.099998,186.699997,7.0,949.900024,847.000000,3,19.930000,14.251810
882,140.000000,133.000000,103.000000,200.000000,7.0,916.000000,753.000000,28,36.439999,31.728161
567,203.500000,135.699997,0.000000,185.699997,0.0,1076.199951,759.299988,7,11.960000,11.809381
923,162.000000,207.000000,172.000000,216.000000,10.0,822.000000,638.000000,28,39.840000,28.816653
...,...,...,...,...,...,...,...,...,...,...
258,212.500000,0.000000,100.400002,159.300003,8.7,1007.799988,903.599976,100,42.919998,44.931956
551,255.000000,0.000000,0.000000,192.000000,0.0,889.799988,945.000000,3,8.200000,6.626353
528,359.000000,19.000000,141.000000,154.000000,10.9,942.000000,801.000000,7,38.610001,38.309511
812,310.000000,0.000000,0.000000,192.000000,0.0,970.000000,850.000000,90,34.680000,34.527364


In [7]:
final_model = finalize_model(model)
save_model(final_model, 'concrete_UCS')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['cement', 'blast_furnace_slag',
                                              'fly_ash', 'water',
                                              'superplasticizer',
                                              'coarse_aggregate',
                                              'fine_aggregate ', 'age'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('actual_estimator',
                  BaggingRegressor(estimator=LGBMRegressor(bagging_fraction=0.8,
                                                           bagging_freq=3,
                 

In [8]:
saved_model = load_model('concrete_UCS')

Transformation Pipeline and Model Successfully Loaded


In [5]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.1518,19.7625,4.4455,0.9288,0.1783,0.1314
1,2.7067,16.0543,4.0068,0.9337,0.1620,0.1171
2,2.8942,24.0939,4.9086,0.9140,0.1861,0.1293
3,2.7206,13.7125,3.7030,0.9558,0.1289,0.0978
4,3.3683,24.4087,4.9405,0.9136,0.1407,0.1054
5,3.3548,21.9338,4.6834,0.9197,0.1563,0.1002
6,2.8803,17.0569,4.1300,0.9356,0.1245,0.0876
7,3.0403,17.0189,4.1254,0.9246,0.1441,0.1097
8,3.9340,40.6537,6.3760,0.8897,0.1866,0.1389


In [9]:
train_pipe = saved_model[:-1].transform(df)

In [12]:
print(saved_model)

Pipeline(memory=FastMemory(location=/var/folders/py/yncddnwd6kn7d9dzfmz1kcbm0000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['cement', 'blast_furnace_slag',
                                             'fly_ash', 'water',
                                             'superplasticizer',
                                             'coarse_aggregate',
                                             'fine_aggregate ', 'age'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transfor...
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 BaggingRegressor(estimator=LGBMRegressor(bagging_fraction=0.8,
                                                          bagging_freq=3,
                     

In [16]:
print(saved_model.named_steps)

{'numerical_imputer': TransformerWrapper(include=['cement', 'blast_furnace_slag', 'fly_ash', 'water',
                            'superplasticizer', 'coarse_aggregate',
                            'fine_aggregate ', 'age'],
                   transformer=SimpleImputer()), 'categorical_imputer': TransformerWrapper(include=[],
                   transformer=SimpleImputer(strategy='most_frequent')), 'clean_column_names': TransformerWrapper(transformer=CleanColumnNames()), 'actual_estimator': BaggingRegressor(estimator=LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                                         feature_fraction=0.5,
                                         learning_rate=0.3,
                                         min_child_samples=26,
                                         min_split_gain=0.8, n_estimators=230,
                                         n_jobs=-1, num_leaves=100,
                                         random_state=1, reg_alpha=0.005,
                   

In [21]:
print(saved_model.named_steps["actual_estimator"])

BaggingRegressor(estimator=LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                                         feature_fraction=0.5,
                                         learning_rate=0.3,
                                         min_child_samples=26,
                                         min_split_gain=0.8, n_estimators=230,
                                         n_jobs=-1, num_leaves=100,
                                         random_state=1, reg_alpha=0.005,
                                         reg_lambda=4),
                 random_state=1)


In [7]:
explainer = shap.TreeExplainer(lightgbm)
shap_values = explainer.shap_values(df.iloc[:,:-1],df.iloc[:,-1])

In [26]:
shap.initjs()
concrete_index = 0
# print(shap_values[concrete_index,:])
# print(df.iloc[concrete_index,:].to_numpy())
shap.force_plot(explainer.expected_value, shap_values[concrete_index,:], df.iloc[concrete_index,:-1])

In [24]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, df.iloc[:,:-1])

In [30]:
shap.plots.beeswarm(explainer.expected_value, shap_values, df.iloc[:,:-1])

TypeError: The beeswarm plot requires an `Explanation` object as the `shap_values` argument.